In [ ]:
from openai import OpenAI

def format_paragraphs_numbered(l):
    to_return = ""
    for i, s in enumerate(l):
        to_return += f"#{i} - \"{s}\"\n"
    return to_return

system_prompt_template = """**Role:** You are an expert clinical information analyst specializing in evaluating text relevance using Mistral models.

**Context:** You will be provided with a clinical question (derived from a patient's query via a patient portal) and a list of numbered paragraphs extracted from a clinical note.

**Goal:** Determine if a *specific* paragraph from the list is relevant to answering the provided clinical question.

**Definition of Relevance:** A paragraph is relevant if it contains information that helps answer, contributes to answering, or is directly related to the topic of the question. It does *not* need to provide the complete answer on its own.

**Task Instructions:**
1.  You will receive the Clinical Question and the full List of Paragraphs first.
2.  Then, you will be asked to evaluate a *specific* paragraph, identified by its number (0-indexed).
3.  Focus your analysis *exclusively* on the content of the specified paragraph number. Do not base your relevance decision on other paragraphs in the list.
4.  Generate a step-by-step Chain of Thought (CoT) reasoning process to justify your decision. Clearly explain *why* the specified paragraph is or is not relevant based on the question's topic and the paragraph's content.
5.  Enclose your entire Chain of Thought reasoning securely within `<think>` and `</think>` tags.
6.  Immediately following the closing `</think>` tag, provide your final answer as *only* "Yes" or "No".

**Output Format:**
<think>
[Your detailed step-by-step reasoning comparing the specific paragraph's content to the question's requirements, focusing only on the specified paragraph.]
</think>
[Yes or No]
"""

client = OpenAI(
    base_url="http://localhost:8888/v1",
    api_key="anything"
)

def predict(question, sentences, sentence_number):
    formatted_note_paragraphs = format_paragraphs_numbered(sentences)

    messages = [
        {
            "role": "system",
            "content": system_prompt_template
        },
        {
            "role": "user",
            "content": f"""Here is the clinical context:

**Clinical Question:**
{question}

**List of Paragraphs (0-indexed):**
{formatted_note_paragraphs}
"""
        },
        {
            "role": "user",
            "content": f"Following the instructions precisely, evaluate if paragraph number {sentence_number} is relevant to answering the Clinical Question. Provide your reasoning within <think> tags followed immediately by Yes or No."
        }
    ]

    response = client.chat.completions.create(
            messages=messages,
            stream=False,
            model="loaded-model",
            temperature=0.1,
            max_tokens=4096,
        )
    
    return response.choices[0].message.content

In [2]:
import json

with open("../v1_test.json", "r", encoding="utf-8") as f:
    data = json.load(f)

In [ ]:
import re

def extract_yes_no_answer(text):
    text = re.sub(r'<think>.*?</think>', '', text, flags=re.DOTALL | re.IGNORECASE)

    matches = re.findall(r'\\text\{([^}]*)\}', text)

    if matches:
        last_found = matches[-1]
        return last_found
    
    matches = re.findall(r'\\boxed\{([^}]*)\}', text)

    if matches:
        last_found = matches[-1]
        return last_found

    text = text.replace("\n", " ")
    text = re.sub(r"\s+", " ", text)
    text = text.strip().lower()
    text = text.replace("*", "")
    text = text.replace(":", "")

    yes_patterns = [
        r'^\s*yes\b',
        r'\byes,\s',
        r'\bthe answer is yes\b',
        r'\bthe final answer is yes\b',
        r'\bit is (very )?likely.*\byes\b',
        r'\byes it is\b',
        r'\banswer yes\b',
    ]
    
    no_patterns = [
        r'^\s*no\b',
        r'\bno,\s',
        r'\bthe answer is no\b',
        r'\bthe final answer is no\b',
        r'\bit is (very )?unlikely.*\bno\b',
        r'\bno it is not\b',
        r'\banswer no\b',
    ]

    for pat in yes_patterns:
        if re.search(pat, text):
            return "Yes"

    for pat in no_patterns:
        if re.search(pat, text):
            return "No"

    return "Unknown"


In [4]:
answer_to_number = {
    "No": 0,
    "Yes": 1,
    "Unknown": -1,
}

predictions_to_save = {}

for i, case in data.items():
    question = case["clinical_question"]
    note_excerpt = case["note_excerpt"]
    sentences = case["note_excerpt_sentences"]
    
    case_to_save = {}
    result_list = []
    for sentence_number, sentence in enumerate(sentences):
        result = predict(question, sentences, sentence_number)
        answer = extract_yes_no_answer(result)
        result_list.append(answer_to_number[answer])
        case_to_save[sentence_number] = {
            "sentence": sentence,
            "raw_prediction": result,
            "prediction": answer_to_number[answer]
        }

    predictions_to_save[i] = case_to_save

    print(i, result_list)

21 [1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
22 [1, 1, 1, 0, 0, 0, 0, 0, 0]
23 [0, 0, 0, 0, 0, 1, 0, 0]
24 [1, 1, 0, 1, 0]
25 [0, 1, 1, 0, 0, 0, 0]
26 [0, 0, 1, 1, 0, 0, 0]
27 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
28 [0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0]
29 [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
30 [0, 0, 0, 0, 0, 1, 1, 1, 1]
31 [0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
32 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
33 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
34 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
35 [0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
36 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
37 [0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0]
38 [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1]
39 [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0]
40 [0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1]
41 [0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1]
42 [

In [ ]:
with open("../predictions-mistral-promptB.json", "w", encoding="utf-8") as f:
    json.dump(predictions_to_save, f, indent=4, ensure_ascii=False)